In [ ]:
!pip install tensorflow==2.16.1

In [ ]:
import tensorflow as tf

print(tf.__version__)

2.16.1


### Import Linbraries and Install Packages

In [ ]:
import keras
import os
from google.colab import userdata
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import matplotlib.pyplot as plt

### Download data and preprocess it

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10

100% 584M/586M [00:27<00:00, 23.5MB/s]
100% 586M/586M [00:27<00:00, 22.4MB/s]


In [ ]:
!unzip "/content/animals10.zip"

Explore the downlaoded dataste

In [ ]:
root_path = '/content/raw-img'

In [ ]:
animals = os.listdir(root_path)
split_size = 0.9

training_dir = os.path.join('/content', 'training')
testing_dir = os.path.join('/content', 'testing')

os.makedirs(training_dir)
os.makedirs(testing_dir)

In [ ]:
for animal in animals:
  training_sub_path = os.path.join(training_dir, animal)
  testing_sub_path = os.path.join(testing_dir, animal)
  os.makedirs(training_sub_path)
  os.makedirs(testing_sub_path)

In [ ]:
for animal in animals:
  curr_path = os.path.join(root_path, animal)
  target_train_dir = os.path.join('/content/training', animal)
  target_test_dir = os.path.join('/content/testing', animal)

  valid_files = []
  file_names = os.listdir(curr_path)

  for file_name in file_names:
    if os.path.getsize(os.path.join(curr_path, file_name)) > 0:
      valid_files.append(file_name)
    else:
      print("bad pic")

  shuffled_files = random.sample(valid_files, len(valid_files))

  split_point = int(len(shuffled_files)*split_size)
  train_files = shuffled_files[:split_point]
  test_files = shuffled_files[split_point:]

  print(len(train_files))
  print(len(test_files))

  for train_file in train_files:
    curr_image_path = os.path.join(curr_path, train_file)
    target_dir_path = os.path.join(target_train_dir, train_file)
    copyfile(curr_image_path, target_dir_path)


  for test_file in test_files:
    curr_image_path = os.path.join(curr_path, test_file)
    target_dir_path = os.path.join(target_test_dir, test_file)
    copyfile(curr_image_path, target_dir_path)

1900
212
2788
310
1679
187
4338
483
1638
182
2360
263
1675
187
1501
167
1301
145
4376
487


In [ ]:
print(len(os.listdir('/content/training/cane')))

4376


In [ ]:
def train_val_generators(training_dir, validation_dir):

  # Instantiate a ImageDataGenerator class with some parameters specifying how to do image agumentaion
  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')


  # now using 'flow_from_directory' method create 'train_generator'
  train_generator = train_datagen.flow_from_directory(
      directory=training_dir,
      batch_size=64,
      class_mode='categorical',
      target_size=(224, 224)
  )

  # we don't do image agumnetaion for valdiation data
  validation_datagen = ImageDataGenerator(rescale=1./255)
  validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                batch_size=64,
                                                                class_mode='categorical',
                                                                target_size=(224, 224))

  return train_generator, validation_generator

In [ ]:
TRAINING_DIR = '/content/training'
VALIDATION_DIR = '/content/testing'
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 23556 images belonging to 10 classes.
Found 2623 images belonging to 10 classes.


### Create the model

In [ ]:
base_model = keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

def create_model(base_model):
  inputs = base_model.input
  x = base_model(inputs)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(512, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  predictions = keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  return model

model = create_model(base_model)
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,857,002 (87.19 MB)

 Trainable params: 22,822,570 (87.06 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
print(model.input_shape)
print(model.output_shape)

(None, 224, 224, 3)
(None, 10)


#### Train

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_accuracy') and logs.get('val_accuracy') > 0.85:
      print("Stop training")
      self.model.stop_training = True

In [ ]:
callback = myCallback()

In [ ]:
history = model.fit(train_generator,
                    epochs=50,
                    callbacks=[callback],
                    validation_data=validation_generator)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


369/369 ━━━━━━━━━━━━━━━━━━━━ 400s 847ms/step - accuracy: 0.6918 - loss: 0.9998 - val_accuracy: 0.1247 - val_loss: 4.0678
Epoch 2/50
369/369 ━━━━━━━━━━━━━━━━━━━━ 271s 720ms/step - accuracy: 0.8097 - loss: 0.6098 - val_accuracy: 0.0999 - val_loss: 12.4438
Epoch 3/50
369/369 ━━━━━━━━━━━━━━━━━━━━ 269s 716ms/step - accuracy: 0.7809 - loss: 0.7043 - val_accuracy: 0.8132 - val_loss: 0.5915
Epoch 4/50
369/369 ━━━━━━━━━━━━━━━━━━━━ 272s 725ms/step - accuracy: 0.8563 - loss: 0.4636 - val_accuracy: 0.7396 - val_loss: 0.8742
Epoch 5/50
369/369 ━━━━━━━━━━━━━━━━━━━━ 0s 716ms/step - accuracy: 0.8778 - loss: 0.3913Stop training
369/369 ━━━━━━━━━━━━━━━━━━━━ 274s 729ms/step - accuracy: 0.8778 - loss: 0.3913 - val_accuracy: 0.8544 - val_loss: 0.4526


In [ ]:
model.save('big_my_model.keras')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv big_my_model.keras /content/drive/MyDrive